# Epidemic spread over the flights network

David Diener (19-733-179)
Kevin Kindler (15-922-529)
Layla Husselman (17-733-130)
Rafael Dubach (18-755-140)


https://ipywidgets.readthedocs.io/en/stable/

In [ ]:
import graph_tool.all as gt
g = gt.collection.ns["openflights"]